In [1]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

In [2]:
model = init_chat_model("openai:gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### ffmpeg is necessary for audio conversion

Why do I need ffmpeg? It is a universal media converter.

In [4]:
import subprocess

def is_ffmpeg_installed():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return True
    except FileNotFoundError:
        return False


if is_ffmpeg_installed():
    print("FFmpeg is installed and available in PATH.")
else:
    print("FFmpeg is NOT installed or not in PATH.")

FFmpeg is installed and available in PATH.


* If you have ffmpeg installed, you are good to go.
* However, if you have it not installed, you should insall it first. It should also be available in your PATH

### Audio converter

In [10]:
# Doing a test on only one example audio file for transcription

from docling.document_converter import DocumentConverter, AudioFormatOption
from docling.datamodel.pipeline_options import AsrPipelineOptions
from docling.datamodel import asr_model_specs
from docling.datamodel.base_models import InputFormat
from docling.pipeline.asr_pipeline import AsrPipeline
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file if present

def transcribe_audio(audio_path: str) -> str:
    """Transcribe audio file using Whisper ASR."""

    print(f"\n🎙️  Transcribing: {Path(audio_path).name}")
    print("   This may take a moment on first run (downloading Whisper model)...")

    # Configure ASR pipeline with Whisper Turbo
    pipeline_options = AsrPipelineOptions()
    pipeline_options.asr_options = asr_model_specs.WHISPER_TURBO

    # Create converter with ASR configuration
    converter = DocumentConverter(
        format_options={
            InputFormat.AUDIO: AudioFormatOption(
                pipeline_cls=AsrPipeline,
                pipeline_options=pipeline_options,
            )
        }
    )

    # Transcribe
    result = converter.convert(Path(audio_path).resolve())

    # Export to markdown with timestamps
    transcript = result.document.export_to_markdown()

    return transcript

def main():
    print("=" * 60)
    print("Audio Transcription with Docling + Whisper")
    print("=" * 60)

    # Audio file to transcribe
    audio_path = "../documents/raw/founder_bio.mp3"

    print(f"\nInput: {audio_path}")

    try:
        # Transcribe
        transcript = transcribe_audio(audio_path)

        # Display results
        print("\n" + "=" * 60)
        print("TRANSCRIPT OUTPUT")
        print("=" * 60)
        print(transcript[:800])  # Show first 800 characters
        if len(transcript) > 800:
            print("\n... (truncated for display)")

        # Save to file
        output_path = f"../documents/processed/examples/output_{Path(audio_path).stem}.md"
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(transcript)

        print(f"\n✓ Full transcript saved to: {output_path}")
        print(f"✓ Total length: {len(transcript)} characters")

        # Parse timestamp information
        lines = transcript.split('\n')
        timestamp_lines = [line for line in lines if '[time:' in line]
        if timestamp_lines:
            print(f"✓ Found {len(timestamp_lines)} timestamped segments")
            print(f"\nExample timestamp format:")
            print(f"  {timestamp_lines[0][:80]}...")

    except FileNotFoundError:
        print("\n✗ Error: FFmpeg not found!")
        print("\nPlease install FFmpeg:")
        print("  Windows (Chocolatey): choco install ffmpeg")
        print("  Windows (Conda):      conda install -c conda-forge ffmpeg")
        print("  macOS:                brew install ffmpeg")
        print("  Linux:                apt-get install ffmpeg")

    except Exception as e:
        print(f"\n✗ Error: {e}")
        print("\nMake sure:")
        print("  1. FFmpeg is installed and in PATH")
        print("  2. Audio file exists and is readable")
        print("  3. Audio format is supported (MP3, WAV, M4A, FLAC)")

In [11]:
main()

2025-11-21 12:25:19,910 - INFO - detected formats: [<InputFormat.AUDIO: 'audio'>]
2025-11-21 12:25:19,918 - INFO - Going to convert document batch...
2025-11-21 12:25:19,922 - INFO - Initializing pipeline for AsrPipeline with options hash 56c2b01f576cf4ca74241045cb5b4bbb
2025-11-21 12:25:19,924 - INFO - artifacts-path: None
2025-11-21 12:25:19,925 - INFO - accelerator_options: num_threads=4 device='auto' cuda_use_flash_attention2=False
2025-11-21 12:25:19,927 - INFO - Accelerator device: 'cpu'
2025-11-21 12:25:19,928 - INFO - Available device for Whisper: cpu
2025-11-21 12:25:19,929 - INFO - loading _NativeWhisperModel(turbo)


Audio Transcription with Docling + Whisper

Input: ../documents/raw/founder_bio.mp3

🎙️  Transcribing: founder_bio.mp3
   This may take a moment on first run (downloading Whisper model)...


2025-11-21 12:25:33,524 - INFO - Processing document founder_bio.mp3
2025-11-21 12:25:33,525 - INFO - start _build_document in AsrPipeline: D:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres\documents\raw\founder_bio.mp3
d:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres\.venv\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English


2025-11-21 12:26:18,963 - INFO - Finished converting document founder_bio.mp3 in 59.06 sec.


[00:00.000 --> 00:10.200]  I am the CEO and co-founder of ABC company and I was born in Berlin and I
[00:10.200 --> 00:19.580]  had around 2 million US dollars for investment in 2023

TRANSCRIPT OUTPUT
[time: 0.0-10.2]  I am the CEO and co-founder of ABC company and I was born in Berlin and I

[time: 10.2-19.58]  had around 2 million US dollars for investment in 2023

✓ Full transcript saved to: ../documents/processed/examples/output_founder_bio.md
✓ Total length: 167 characters
✓ Found 2 timestamped segments

Example timestamp format:
  [time: 0.0-10.2]  I am the CEO and co-founder of ABC company and I was born in B...


### Text converter

In [14]:
# Doing a test on three sample text docs

def process_text_docs(file_path: str) -> dict:
    """Process a single document and return metadata."""
    try:
        print(f"\n📄 Processing: {Path(file_path).name}")

        # instantiate the converter
        converter = DocumentConverter()

        # Convert document
        result = converter.convert(file_path)

        # Export to markdown
        markdown = result.document.export_to_markdown()

        # Get document info
        doc_info = {
            'file': Path(file_path).name,
            'format': Path(file_path).suffix,
            'status': 'Success',
            'markdown_length': len(markdown),
            'preview': markdown[:200].replace('\n', ' ')
        }

        # Save output
        output_file = f"../documents/processed/examples/output_{Path(file_path).stem}.md"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(markdown)

        doc_info['output_file'] = output_file

        print(f"   ✓ Converted successfully")
        print(f"   ✓ Output: {output_file}")

        return doc_info

    except Exception as e:
        print(f"   ✗ Error: {e}")
        return {
            'file': Path(file_path).name,
            'format': Path(file_path).suffix,
            'status': 'Failed',
            'error': str(e)
        }

def main():
    print("=" * 60)
    print("Multi-Format Document Processing with Docling")
    print("=" * 60)

    # List of documents to process
    documents = [
        "../documents/raw/technical-architecture-guide.pdf",
        "../documents/raw/meeting-notes-2025-01-08.docx",
        "../documents/raw/company-overview.md",
    ]

    # Process all documents
    results = []
    for doc_path in documents:
        result = process_text_docs(doc_path)
        results.append(result)

    # Summary
    print("\n" + "=" * 60)
    print("CONVERSION SUMMARY")
    print("=" * 60)

    for result in results:
        status_icon = "✓" if result['status'] == 'Success' else "✗"
        print(f"{status_icon} {result['file']} ({result['format']})")
        if result['status'] == 'Success':
            print(f"   Length: {result['markdown_length']} chars")
            print(f"   Preview: {result['preview']}...")
        else:
            print(f"   Error: {result.get('error', 'Unknown')}")
        print()

    success_count = sum(1 for r in results if r['status'] == 'Success')
    print(f"Converted {success_count}/{len(results)} documents successfully")

In [15]:
main()

2025-11-21 12:33:21,015 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]


Multi-Format Document Processing with Docling

📄 Processing: technical-architecture-guide.pdf


2025-11-21 12:33:23,988 - INFO - Going to convert document batch...
2025-11-21 12:33:23,991 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4f2edc0f7d9bb60b38ebfecf9a2609f5
2025-11-21 12:33:24,099 - INFO - Loading plugin 'docling_defaults'
2025-11-21 12:33:24,122 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-21 12:33:24,175 - INFO - Loading plugin 'docling_defaults'
2025-11-21 12:33:24,227 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-21 12:33:24,556 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-11-21 12:33:24,559 - INFO - easyocr cannot be used because it is not installed.
2025-11-21 12:33:25,832 - INFO - Accelerator device: 'cpu'
[INFO] 2025-11-21 12:33:25,874 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-21 12:33:25,945 [RapidOCR] download_file.py:60: File exists and is valid: D:\Documents\Learning\Software_Engineering\RAG\langchai

   ✓ Converted successfully
   ✓ Output: ../documents/processed/examples/output_technical-architecture-guide.md

📄 Processing: meeting-notes-2025-01-08.docx


[INFO] 2025-11-21 12:34:02,020 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-21 12:34:02,023 [RapidOCR] download_file.py:60: File exists and is valid: D:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres\.venv\Lib\site-packages\rapidocr\models\ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-11-21 12:34:02,024 [RapidOCR] torch.py:54: Using D:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres\.venv\Lib\site-packages\rapidocr\models\ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-11-21 12:34:02,117 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-21 12:34:02,184 [RapidOCR] download_file.py:60: File exists and is valid: D:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-11-21 12:34:02,185 [RapidOCR] torch.py:54: Using D:\Documents\Learning\Software_Engineering\RAG\langchain_docling_postgres\.venv\Lib\site-packages\rapido

   ✓ Converted successfully
   ✓ Output: ../documents/processed/examples/output_meeting-notes-2025-01-08.md

📄 Processing: company-overview.md


2025-11-21 12:34:33,976 - INFO - Finished converting document company-overview.md in 0.40 sec.


   ✓ Converted successfully
   ✓ Output: ../documents/processed/examples/output_company-overview.md

CONVERSION SUMMARY
✓ technical-architecture-guide.pdf (.pdf)
   Length: 8699 chars
   Preview: ## Technical Architecture Guide  NeuralFlow AI Platform v2.0  Document Version: 2.3 | Last Updated: December 15, 2024  Classification: Internal - Engineering Team  ## 1. System Overview  The NeuralFlo...

✓ meeting-notes-2025-01-08.docx (.docx)
   Length: 9338 chars
   Preview: Meeting:  Product Strategy &amp; Roadmap Review  Date:  January 8, 2025  Time:  2:00 PM - 4:00 PM PT  Location:  Conference Room A  Attendees:  Sarah Chen (CTO), Michael Zhang (VP Product), Jennifer M...

✓ company-overview.md (.md)
   Length: 3102 chars
   Preview: # NeuralFlow AI - Company Overview  ## About Us  NeuralFlow AI is a cutting-edge AI automation agency founded in 2023, specializing in intelligent workflow automation, natural language processing solu...

Converted 3/3 documents successfully


### Bring everything together

In [ ]:
# First, converter for text documents
def process_text_docs(raw_file_path: str):
    """Process a single document and return metadata."""
    try:
        print(f"\n📄 Processing: {Path(raw_file_path).name}")

        # instantiate the converter
        converter = DocumentConverter()

        # Convert document
        result = converter.convert(raw_file_path)

        # Export to markdown
        markdown = result.document.export_to_markdown()

        # Save output
        output_file = f"../documents/processed/{Path(raw_file_path).stem}.md"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(markdown)

        print(f"   ✓ Converted successfully")
        print(f"   ✓ Output: {output_file}")
    
    except Exception as e:
        print(f"   ✗ Error: {e}")
        return {
            'file': Path(raw_file_path).name,
            'format': Path(raw_file_path).suffix,
            'status': 'Failed',
            'error': str(e)
        }

# Now, function for audio transcription    
def process_audio_docs(audio_path: str):
    """Transcribe audio file using Whisper ASR."""

    print(f"\n🎙️  Transcribing: {Path(audio_path).name}")
    print("   This may take a moment on first run (downloading Whisper model)...")
    print(f"\nInput: {audio_path}")

    # Configure ASR pipeline with Whisper Turbo
    pipeline_options = AsrPipelineOptions()
    pipeline_options.asr_options = asr_model_specs.WHISPER_TURBO

    # Create converter with ASR configuration
    converter = DocumentConverter(
        format_options={
            InputFormat.AUDIO: AudioFormatOption(
                pipeline_cls=AsrPipeline,
                pipeline_options=pipeline_options,
            )
        }
    )

    # Transcribe
    result = converter.convert(Path(audio_path).resolve())

    # Export to markdown with timestamps
    transcript = result.document.export_to_markdown()

    try:
        # Display results
        print("TRANSCRIPT OUTPUT:")
        print(transcript[:800])  # Show first 800 characters
        if len(transcript) > 800:
            print("\n... (truncated for display)")

        # Save to file
        output_path = f"../documents/processed/{Path(audio_path).stem}.md"
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(transcript)

        print(f"\n✓ Full transcript saved to: {output_path}")
        print(f"✓ Total length: {len(transcript)} characters")

        # Parse timestamp information
        lines = transcript.split('\n')
        timestamp_lines = [line for line in lines if '[time:' in line]
        if timestamp_lines:
            print(f"✓ Found {len(timestamp_lines)} timestamped segments")
            print(f"\nExample timestamp format:")
            print(f"  {timestamp_lines[0][:80]}...")

    except FileNotFoundError:
        print("\n✗ Error: FFmpeg not found!")
        print("\nPlease install FFmpeg:")
        print("  Windows (Chocolatey): choco install ffmpeg")
        print("  Windows (Conda):      conda install -c conda-forge ffmpeg")
        print("  macOS:                brew install ffmpeg")
        print("  Linux:                apt-get install ffmpeg")

    except Exception as e:
        print(f"\n✗ Error: {e}")
        print("\nMake sure:")
        print("  1. FFmpeg is installed and in PATH")
        print("  2. Audio file exists and is readable")
        print("  3. Audio format is supported (MP3, WAV, M4A, FLAC)")

def process_raw_documents(raw_documents_dir: str):
    print("=" * 60)
    print("Multi-Format Document Processing with Docling")
    print("=" * 60)

    # List of documents to process
    directory = Path(raw_documents_dir)
    documents = [str(file) for file in directory.iterdir() if file.is_file()]

    # Process all documents
    results = []
    for doc_path in documents:
        if Path(doc_path).suffix == '.mp3':
            result = process_audio_docs(doc_path)
        else:
            result = process_text_docs(doc_path)
        results.append(result)

In [ ]:
process_raw_documents("../documents/raw")